In [1]:
import pymysql
import csv
import pandas as pd

In [2]:
file = '/users/eiofinova/Downloads/GTEx4GenesonPPIonly.txt'

relations = pd.read_csv(file, sep='\t')
relations

,Name,Description,Adipose - Subcutaneous,Adipose - Visceral (Omentum),Adrenal Gland,Artery - Aorta,Artery - Coronary,Artery - Tibial,Bladder,Brain - Amygdala,...,Skin - Not Sun Exposed (Suprapubic),Skin - Sun Exposed (Lower leg),Small Intestine - Terminal Ileum,Spleen,Stomach,Testis,Thyroid,Uterus,Vagina,Whole Blood
0,ENSG00000187634.11,SAMD11,0.476230,2.931190,3.78499,5.717060,3.104580,7.931910,19.05830,0.532897,...,0.531438,0.793609,4.90961,17.48400,3.42340,6.86414,5.41375,11.293200,6.69566,0.046089
1,ENSG00000188976.10,NOC2L,52.861400,48.187700,37.17370,55.491600,51.901500,62.564000,71.86850,18.344900,...,83.104800,90.319900,49.56680,56.15350,42.38180,98.66560,66.78740,77.388400,71.25470,14.988900
2,ENSG00000187961.13,KLHL17,9.678200,8.967750,7.89155,13.917400,13.392400,10.740400,17.60890,3.048490,...,16.258800,16.516900,12.48690,21.23980,10.36080,30.79750,15.22820,17.991200,18.20740,3.581200
3,ENSG00000187583.10,PLEKHN1,0.559729,0.648233,1.92579,0.744398,0.868276,0.717782,4.22114,0.107346,...,72.613400,67.402600,1.40783,2.03468,1.13771,1.37933,1.31803,0.722009,45.33660,0.496924
4,ENSG00000188290.10,HES4,21.789300,24.187600,2.91384,46.628900,64.610400,98.388500,33.05170,16.737000,...,21.685900,17.949000,9.04615,16.31330,9.41045,10.00340,23.28010,39.535100,32.07090,1.205480
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16338,ENSG00000212907.2,MT-ND4L,14087.500000,17101.500000,24185.00000,6621.270000,8125.240000,7443.650000,16004.00000,23020.700000,...,12200.000000,11012.100000,16621.70000,7426.31000,19509.80000,13737.90000,14719.70000,10149.100000,8543.49000,1234.540000
16339,ENSG00000198886.2,MT-ND4,31411.900000,36698.000000,54001.30000,15487.000000,18822.500000,17130.300000,35145.50000,56265.900000,...,27967.200000,24963.900000,37438.90000,22144.10000,44214.70000,30592.50000,32083.10000,22680.900000,18944.90000,3881.600000
16340,ENSG00000198786.2,MT-ND5,4000.320000,4669.680000,6584.28000,4365.330000,4442.610000,3800.300000,6395.82000,9126.800000,...,7146.040000,6442.180000,7910.63000,3380.79000,8208.02000,2601.29000,3525.29000,3686.450000,2739.59000,709.967000
16341,ENSG00000198695.2,MT-ND6,3517.220000,3598.090000,6637.79000,6374.490000,4972.730000,4635.000000,8275.20000,9631.770000,...,8919.460000,8113.220000,9764.49000,5477.36000,9628.75000,1676.59000,2520.41000,3644.000000,2237.43000,960.291000


In [7]:
relations.columns[2:]

Index(['Adipose - Subcutaneous', 'Adipose - Visceral (Omentum)',
       'Adrenal Gland', 'Artery - Aorta', 'Artery - Coronary',
       'Artery - Tibial', 'Bladder', 'Brain - Amygdala',
       'Brain - Anterior cingulate cortex (BA24)',
       'Brain - Caudate (basal ganglia)', 'Brain - Cerebellar Hemisphere',
       'Brain - Cerebellum', 'Brain - Cortex', 'Brain - Frontal Cortex (BA9)',
       'Brain - Hippocampus', 'Brain - Hypothalamus',
       'Brain - Nucleus accumbens (basal ganglia)',
       'Brain - Putamen (basal ganglia)', 'Brain - Spinal cord (cervical c-1)',
       'Brain - Substantia nigra', 'Breast - Mammary Tissue',
       'Cells - Cultured fibroblasts', 'Cells - EBV-transformed lymphocytes',
       'Cervix - Ectocervix', 'Cervix - Endocervix', 'Colon - Sigmoid',
       'Colon - Transverse', 'Esophagus - Gastroesophageal Junction',
       'Esophagus - Mucosa', 'Esophagus - Muscularis', 'Fallopian Tube',
       'Heart - Atrial Appendage', 'Heart - Left Ventricle', 'Kidney 

In [3]:
import db_config
dbconf = db_config.asimov_admin
db = pymysql.connect(dbconf["host"], dbconf["user"],
                         dbconf["password"], db='ppi')
cursor = db.cursor()

In [30]:
cursor.close()
db.close()

In [22]:
namespace = 'ppi'

In [23]:
def add_tissue(attribute, attribute_namespace, db_namespace):
    queries = []
    query = "DROP TABLE IF EXISTS new_attrs"
    print(query)
    queries.append(query)
    query = "CREATE TABLE new_attrs (name varchar(255))"
    print(query)
    queries.append(query)
    query = "INSERT INTO new_attrs VALUES (\"%s\")" % '"),("'.join(attribute)
    print(query)
    queries.append(query)
    query = """
    INSERT INTO attributes(external_id, namespace, name, description)
    SELECT new_attrs.name, '%s', new_attrs.name, new_attrs.name
    FROM new_attrs
    LEFT JOIN attributes attrs ON attrs.external_id = new_attrs.name
        AND attrs.namespace = '%s'
    WHERE attrs.id IS NULL
    """ % (attribute_namespace, attribute_namespace)
    print(query)
    queries.append(query)
    query = """
    INSERT INTO attribute_taxonomies(parent_id, child_id, distance, namespace)
    SELECT attrs.id, attrs.id, 1, attrs.namespace
    FROM attributes attrs
    JOIN new_attrs on new_attrs.name = attrs.external_id
    LEFT JOIN attribute_taxonomies at ON at.parent_id = attrs.id
        AND at.child_id = attrs.id AND at.namespace = attrs.namespace
        WHERE at.id IS NULL
    """
    print(query)
    queries.append(query)
    query = "DROP TABLE IF EXISTS new_attrs"
    print(query)
    queries.append(query)

    for query in queries:
        cursor.execute(query)
    db.commit()

add_tissue(relations.columns[2:], 'GTEx', 'ppi')



DROP TABLE IF EXISTS new_attrs
CREATE TABLE new_attrs (name varchar(255))
INSERT INTO new_attrs VALUES ("Adipose - Subcutaneous"),("Adipose - Visceral (Omentum)"),("Adrenal Gland"),("Artery - Aorta"),("Artery - Coronary"),("Artery - Tibial"),("Bladder"),("Brain - Amygdala"),("Brain - Anterior cingulate cortex (BA24)"),("Brain - Caudate (basal ganglia)"),("Brain - Cerebellar Hemisphere"),("Brain - Cerebellum"),("Brain - Cortex"),("Brain - Frontal Cortex (BA9)"),("Brain - Hippocampus"),("Brain - Hypothalamus"),("Brain - Nucleus accumbens (basal ganglia)"),("Brain - Putamen (basal ganglia)"),("Brain - Spinal cord (cervical c-1)"),("Brain - Substantia nigra"),("Breast - Mammary Tissue"),("Cells - Cultured fibroblasts"),("Cells - EBV-transformed lymphocytes"),("Cervix - Ectocervix"),("Cervix - Endocervix"),("Colon - Sigmoid"),("Colon - Transverse"),("Esophagus - Gastroesophageal Junction"),("Esophagus - Mucosa"),("Esophagus - Muscularis"),("Fallopian Tube"),("Heart - Atrial Appendage"),("He

In [5]:
queries = ["DROP TABLE IF EXISTS gtex_values",
           "CREATE TABLE gtex_values (node varchar(255), attribute VARCHAR(255), value double)"]
for query in queries:)
    cursor.execute(query)
for column in relations.columns[2:]:
    query = """
    INSERT INTO gtex_values (node, attribute, value)
    VALUES %s
    """ % ",".join(['("%s", "%s", %f)' % (x[0], column, x[1]) for x in relations[["Description", column]].values])
    cursor.execute(query)
query = '''
    INSERT INTO nodes_attributes(node_id, attribute_id, value)
    SELECT nodes.id, attrs.id, value
    FROM attributes attrs
    JOIN gtex_values gtex on gtex.attribute = attrs.external_id
    JOIN nodes ON nodes.symbol = gtex.node
    LEFT JOIN nodes_attributes na ON na.node_id = nodes.id
        AND na.attribute_id = attrs.id
        WHERE na.id IS NULL
'''
#print(query)
cursor.execute(query)
db.commit()

DROP TABLE IF EXISTS gtex_values
CREATE TABLE gtex_values (node varchar(255), attribute VARCHAR(255), value double)

    INSERT INTO nodes_attributes(node_id, attribute_id)
    SELECT nodes.id, attrs.id
    FROM attributes attrs
    JOIN gtex_values gtex on gtex.attribute = attrs.external_id
    JOIN nodes ON nodes.symbol = gtex.node
    LEFT JOIN nodes_attributes na ON na.node_id = nodes.id
        AND na.attribute_id = attrs.id
        WHERE na.id IS NULL



In [35]:
db.commit()